# Market_Analytics - Budget Reallocation Strategy

## Budget Optimization Objective

The goal of this notebook is to **reallocate a fixed marketing budget** across campaigns to maximize expected conversions,
using **observed historical efficiency**.

### Key Constraints
- Total budget is fixed (no increase in spend)
- No direct ad spend or true revenue data is available
- Optimization is based on *relative performance*, not absolute ROI
- Results provide directional decision support, not financial guarantees

This reflects a **real-world consulting scenario** under partial observability.

In [11]:
# Step 1: Load Campaign-Level Data
import pandas as pd
import numpy as np

# Load campaign performance data
campaign_perf = pd.read_csv(
    "../data/processed/campaign_performance_summary.csv"
)

campaign_perf.head()

,Ad Type,Ad Placement,Ad Topic,Location,impressions,clicks,expected_conversions,cost,revenue,CTR,CVR,CPA,ROI,ROI_Rank,CPA_Rank
0,Banner,Search Engine,Fashion,Rural,6323.065886,229,40.8934,114.5,1635.736,0.036217,0.178574,2.799963,13.285904,198.0,198.0
1,Banner,Search Engine,Fashion,Suburban,6259.956920,271,54.1309,135.5,2165.236,0.043291,0.199745,2.503191,14.979601,116.0,116.0
2,Banner,Search Engine,Fashion,Urban,6822.505967,278,48.5637,139.0,1942.548,0.040747,0.174690,2.862220,12.975165,202.0,202.0
3,Banner,Search Engine,Finance,Rural,6990.525018,265,51.5156,132.5,2060.624,0.037908,0.194398,2.572036,14.551879,141.0,141.0
4,Banner,Search Engine,Finance,Suburban,5128.252635,234,50.6310,117.0,2025.240,0.045630,0.216372,2.310837,16.309744,47.0,47.0


In [12]:
# Step 2: Create CVR, CPC, conversions_per_dollar 
# Safety checks
required_cols = ["clicks", "expected_conversions", "cost"]
missing = set(required_cols) - set(campaign_perf.columns)

if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Conversion rate (CVR)
campaign_perf["conversion_rate"] = (
    campaign_perf["expected_conversions"] / campaign_perf["clicks"]
).replace([np.inf, -np.inf], np.nan)

# Cost per click (CPC)
campaign_perf["cpc"] = (
    campaign_perf["cost"] / campaign_perf["clicks"]
).replace([np.inf, -np.inf], np.nan)

# Conversions per dollar (PRIMARY OPTIMIZATION METRIC)
campaign_perf["conversions_per_dollar"] = (
    campaign_perf["expected_conversions"] / campaign_perf["cost"]
).replace([np.inf, -np.inf], np.nan)

# Historical spend share
campaign_perf["spend_share"] = (
    campaign_perf["cost"] / campaign_perf["cost"].sum()
)

campaign_perf[
    ["conversion_rate", "cpc", "conversions_per_dollar", "spend_share"]
].describe()

,conversion_rate,cpc,conversions_per_dollar,spend_share
count,216.000000,216.0,216.000000,216.000000
mean,0.202929,0.5,0.405858,0.004630
std,0.019365,0.0,0.038731,0.000721
min,0.163405,0.5,0.326809,0.002882
25%,0.189291,0.5,0.378582,0.004130
50%,0.201528,0.5,0.403055,0.004602
75%,0.213719,0.5,0.427437,0.005074
max,0.266215,0.5,0.532429,0.006818


In [13]:
# Step 3: Budget Reallocation Logic
TOTAL_BUDGET = campaign_perf["cost"].sum()

# Normalize efficiency scores
campaign_perf["efficiency_weight"] = (
    campaign_perf["conversions_per_dollar"] /
    campaign_perf["conversions_per_dollar"].sum()
)

# Constraints to avoid killing or over-funding campaigns
MIN_SHARE = 0.05
MAX_SHARE = 0.40

campaign_perf["optimized_spend_share"] = (
    campaign_perf["efficiency_weight"]
    .clip(lower=MIN_SHARE, upper=MAX_SHARE)
)

# Re-normalize after clipping
campaign_perf["optimized_spend_share"] /= (
    campaign_perf["optimized_spend_share"].sum()
)

# Allocate optimized budget
campaign_perf["optimized_budget"] = (
    campaign_perf["optimized_spend_share"] * TOTAL_BUDGET
)


In [14]:
# Step 4: Estimate Optimized Click Volume
campaign_perf["expected_clicks_optimized"] = (
    campaign_perf["optimized_budget"] / campaign_perf["cpc"]
)

In [15]:
# Step 5: Estimate Optimized Conversions
campaign_perf["expected_conversions_optimized"] = (
    campaign_perf["expected_clicks_optimized"] *
    campaign_perf["conversion_rate"]
)

In [16]:
# Step 6: Conversion Lift Analysis
summary = campaign_perf[[
    "expected_conversions",
    "expected_conversions_optimized"
]].sum()

lift = (
    summary["expected_conversions_optimized"]
    - summary["expected_conversions"]
)

lift_pct = lift / summary["expected_conversions"] * 100

print(f"Expected Conversion Lift: {lift:.2f}")
print(f"Expected Lift Percentage: {lift_pct:.2f}%")

Expected Conversion Lift: -6.07
Expected Lift Percentage: -0.06%


In [17]:
# Step 7: Executive Baseline vs Optimized Summary
summary_table = pd.DataFrame({
    "Metric": ["Total Budget", "Total Expected Conversions"],
    "Baseline": [
        campaign_perf["cost"].sum(),
        campaign_perf["expected_conversions"].sum()
    ],
    "Optimized": [
        campaign_perf["optimized_budget"].sum(),
        campaign_perf["expected_conversions_optimized"].sum()
    ]
})

summary_table

,Metric,Baseline,Optimized
0,Total Budget,25153.0000,25153.000000
1,Total Expected Conversions,10214.6096,10208.536025


In [18]:
# Step 8: Diagnose Winners & Losers
campaign_perf["budget_change"] = (
    campaign_perf["optimized_budget"] - campaign_perf["cost"]
)

diagnostic = campaign_perf[[
    "cost",
    "optimized_budget",
    "budget_change",
    "conversion_rate",
    "cpc",
    "conversions_per_dollar"
]].sort_values("budget_change", ascending=False)

diagnostic.head(10)

,cost,optimized_budget,budget_change,conversion_rate,cpc,conversions_per_dollar
145,72.5,116.449074,43.949074,0.212688,0.5,0.425375
95,77.0,116.449074,39.449074,0.201735,0.5,0.403470
140,80.0,116.449074,36.449074,0.228156,0.5,0.456313
100,80.5,116.449074,35.949074,0.216472,0.5,0.432944
195,82.5,116.449074,33.949074,0.217384,0.5,0.434767
163,84.0,116.449074,32.449074,0.175000,0.5,0.350000
160,85.5,116.449074,30.949074,0.218124,0.5,0.436248
189,86.0,116.449074,30.449074,0.187397,0.5,0.374794
29,86.5,116.449074,29.949074,0.196633,0.5,0.393266
116,87.0,116.449074,29.449074,0.174246,0.5,0.348492


In [19]:
# Step 9: Efficiency Frontier Insight
campaign_perf["conversion_efficiency_rank"] = (
    campaign_perf["conversions_per_dollar"]
    .rank(ascending=False)
)

campaign_perf[
    ["cost", "optimized_budget", "conversions_per_dollar", "conversion_efficiency_rank"]
].sort_values("conversion_efficiency_rank").head(10)

,cost,optimized_budget,conversions_per_dollar,conversion_efficiency_rank
214,93.0,116.449074,0.532429,1.0
27,126.0,116.449074,0.502916,2.0
7,94.5,116.449074,0.501817,3.0
121,121.5,116.449074,0.500874,4.0
124,143.0,116.449074,0.493506,5.0
212,123.0,116.449074,0.492270,6.0
97,93.5,116.449074,0.487843,7.0
115,112.0,116.449074,0.486341,8.0
201,124.5,116.449074,0.485760,9.0
71,106.0,116.449074,0.484465,10.0


In [21]:
# Step 10: Save Outputs
campaign_perf.to_csv(
    "../reports/budget_optimization_results.csv",
    index=False
)

### Executive Interpretation

Budget reallocation based on observed conversion efficiency per dollar resulted in minimal aggregate lift (-0.06%), indicating that the historical budget allocation was already close to an efficient frontier under current cost and conversion dynamics. Optimization primarily redistributes spend toward marginally more efficient campaigns rather than generating large incremental gains. This outcome reflects realistic constraints in mature marketing programs and supports incremental budget refinement rather than radical reallocation.

Limitations & Assumptions

- Conversion rates are assumed stable under moderate budget changes
- No diminishing returns or channel saturation modeled
- CPC assumed constant within campaign segments
- No true revenue or profit data available
- Results represent decision guidance, not guaranteed financial outcomes

This mirrors **real-world marketing optimization under partial observability**.